### 패키치 설치

In [ ]:
!pip install numpy pandas selenium bs4 requests tqdm

### 라이브러리 임포트

In [40]:
import pandas as pd
import numpy as np

# chromedriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException


import requests
import time
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
from datetime import datetime
import os

### 웹드라이버 창 열기

In [92]:
service = Service("/Users/bykwon/Downloads/chromedriver-mac-arm64 3/chromedriver") # 여러분의 chromedriver 경로로 수정해주세요!
driver = webdriver.Chrome(service=service)
driver.maximize_window()

In [93]:
# 대상 URL로 이동
url = "https://gift.kakao.com/page/code/life_funny"
driver.get(url)

In [ ]:
# 무한 스크롤을 통한 데이터 로드
target_count = 100  # 크롤링하려는 상품 수
scroll_pause_time = 0.5

# 크롤링된 상품 링크를 저장할 리스트
product_links = set()

while len(product_links) < target_count:
    # 현재 페이지 HTML 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # id가 "promotion_prdList_3"로 시작하는 섹션 탐색
    sections = soup.find_all("div", id=re.compile("^promotion_prdList_3"))
    for section in sections:
        items = section.find_all("a", class_="link_thumb")
        for item in items:
            link = item.get("href")
            if link not in product_links:
                product_links.add('https://gift.kakao.com'+link)

    # 목표 수량 도달 시 루프 종료
    if len(product_links) >= target_count:
        break

    # 스크롤 다운
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(scroll_pause_time)

    # 상품 수 출력 (디버깅용)
    print(f"중복 없는 상품 수: {len(product_links)}")

# 결과 출력
print("수집된 상품 링크:")
for link in product_links:
    print(link)
print(f"총 {len(product_links)}개의 상품 링크 수집 완료")

### 필요한 함수 정의

In [9]:
def crawl_info(link):
    # 동적 크롤링을 위해 selenium으로 페이지를 엽니다.
    driver.get(link)
    time.sleep(2)

    # 페이지 소스를 읽어 BeautifulSoup으로 파싱합니다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 상품 정보를 담을 딕셔너리를 생성합니다.
    # product_info = {}

    # 상품 정보를 추출합니다.
    name = soup.select_one('h2.tit_subject').text.strip()
    num_review = soup.select_one('span.txt_desc').text.strip()
    num_review = re.sub(r'[^\d]', '', num_review)
    price = soup.select_one('span.txt_total').text.strip()
    price = re.sub(r'[^\d]', '', price)
    heart = soup.select_one('span.count_wishes').text.strip()
    heart = re.sub(r'[^\d]', '', heart)
    brand = soup.select_one('span.txt_shopname').text.strip()


    return {'name': name, 'num_review': num_review, 'price': price, 'heart': heart, 'brand': brand}

In [14]:
def crawl_review(link):
    cutoff = datetime.strptime("2024-09-30", "%Y-%m-%d")

    reviews = []

    if driver.current_url != link:
        driver.get(link)

    time.sleep(1)
    
    driver.find_element(value="/html/body/app-root/app-view-wrapper/div/div/main/article/app-home/div/div[1]/app-contents/app-product-detail-tab/div/ul/li[2]/a/span", by=By.XPATH).click()
    time.sleep(1)
    try:
        driver.find_element(value="/html/body/app-root/app-view-wrapper/div/div/main/article/app-home/div/div[1]/app-contents/app-product-detail-tab/div/app-info-review/div/div/div[1]/ul/li[1]/a", by=By.XPATH).click()
        time.sleep(1)
    except:
        print("리뷰 없음")
        return None, reviews
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 만족도 정보 크롤링
    try:
        satisfaction = soup.select_one('em.txt_point span.txt_percent').text.strip()
    except:
        satisfaction = None

    # 리뷰 정보 크롤링
    reviews = []
    cnt = 0
    flag = 0

    while True:
        # 페이지 HTML 가져오기
        html = driver.page_source
        time.sleep(1)
        soup = BeautifulSoup(html, 'html.parser')

        # 리뷰 항목 선택
        review_items = soup.select('app-view-review-item')  # 각 리뷰를 감싸는 요소
        if review_items == []:
            return satisfaction, reviews
        review_items = review_items[cnt:]
        

        for review in review_items:
            date_text = review.select('.txt_reviewinfo')[1].text.strip()
            review_date = datetime.strptime(date_text, "%Y.%m.%d")

            # 날짜 기준 확인
            if review_date < cutoff:
                flag = 1
                break

            # 별점 추출
            rating_text = review.select_one('.area_score').text.strip()
            rating = rating_text.split(' ')[-1]  # "별점 4점 만점 중 4점" -> "4점"

            # 내용 추출
            content = review.select_one('.txt_review').text.strip()

            who = review.select('.txt_hash')[0].text.strip()
            what = review.select('.txt_hash')[1].text.strip()

            # 리뷰 데이터 저장
            reviews.append({
                "datetime": review_date,
                "rating": rating,
                "content": content,
                "who": who,
                "what": what
            })

        # 다음 페이지 이동 전 저장된 리뷰 수 확인 > 다음에 새로 로딩될 때에는 이전 페이지의 리뷰가 중복되지 않도록
        print(f"리뷰 수: {len(reviews)}")
        cnt = len(reviews)
        if flag == 1 or reviews[-1]['datetime'] <= cutoff:
            break
        print(reviews[-1]['datetime'], cutoff)


        # 다음 페이지로 이동
        try:
            driver.find_element(value="/html/body/app-root/app-view-wrapper/div/div/main/article/app-home/div/div[1]/app-contents/app-product-detail-tab/div/app-info-review/div/div/div[2]/button", by=By.XPATH).click()
        except:
            print("리뷰 페이지 마지막")
            break

    return satisfaction, reviews
    
    # <span class="txt_gage">90%</span>
    # /html/body/app-root/app-view-wrapper/div/div/main/article/app-home/div/div[1]/app-contents/app-product-detail-tab/div/app-info-review/div/div/div[2]/button
    # /html/body/app-root/app-view-wrapper/div/div/main/article/app-home/div/div[1]/app-contents/app-product-detail-tab/div/app-info-review/div/div/div[2]/button

### 폴더 생성

In [87]:
os.makedirs("./웃긴선물", exist_ok=True)

### 실행

In [88]:
info_dic = {}
i = 0
for link in tqdm(product_links):
    time.sleep(2)
    # info 
    try:
        info = crawl_info(link) 
    except Exception as e:
        print(e)
        # i += 1
        continue
    time.sleep(1)
    # review
    satisfaction, review = crawl_review(link)

    # 데이터 저장
    info['satisfaction'] = satisfaction
    info['link'] = link


    # 리뷰가 없는 상품은 건너뜁니다.
    if info['num_review'] == '0' or info['num_review'] == 0:
        # i += 1
        continue


    # info_dic에 상품 정보 저장
    info_dic[info['name']] = info

    # 각 상품별로 리뷰 모음 딕셔너리가 생성됩니다.
    # 저장 -> csv
    review_df = pd.DataFrame(review)
    review_df['product'] = info['name']

    review_df.to_csv(f"./웃긴선물/{i}.csv", index=False)
    i += 1

    # 100개 크롤링 후 종료
    if len(info_dic) >= 100:
        break

info_df = pd.DataFrame(info_dic).T
info_df.to_csv("./웃긴선물/metadata.csv", index=False)

  1%|          | 1/94 [00:09<15:14,  9.83s/it]

리뷰 수: 4
리뷰 수: 20
2024-12-30 00:00:00 2024-09-30 00:00:00
리뷰 수: 20
2024-12-30 00:00:00 2024-09-30 00:00:00
리뷰 수: 40
2024-12-11 00:00:00 2024-09-30 00:00:00
리뷰 수: 60
2024-11-20 00:00:00 2024-09-30 00:00:00
리뷰 수: 80
2024-10-25 00:00:00 2024-09-30 00:00:00
리뷰 수: 100
2024-10-08 00:00:00 2024-09-30 00:00:00


  2%|▏         | 2/94 [00:27<22:31, 14.70s/it]

리뷰 수: 110


  3%|▎         | 3/94 [00:37<18:50, 12.42s/it]

...

리뷰 수: 16
리뷰 수: 20
2024-11-07 00:00:00 2024-09-30 00:00:00
리뷰 수: 20
2024-11-07 00:00:00 2024-09-30 00:00:00


 99%|█████████▉| 93/94 [26:30<00:13, 13.90s/it]

리뷰 수: 25


100%|██████████| 94/94 [26:40<00:00, 17.02s/it]

리뷰 수: 6


### (탭별로)

In [110]:
# 링크 수집
# 무한 스크롤을 통한 데이터 로드
target_count = 30 # 크롤링하려는 상품 수
scroll_pause_time = 0.5

# 크롤링된 상품 링크를 저장할 리스트
product_links = set()

while len(product_links) < target_count:
    # 현재 페이지 HTML 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # id가 "promotion_prdList_5"로 시작하는 섹션 탐색
    sections = soup.find_all("div", id=re.compile("^promotion_prdList_210"))
    for section in sections:
        items = section.find_all("a", class_="link_thumb")
        for item in items:
            link = item.get("href")
            if link not in product_links:
                product_links.add('https://gift.kakao.com'+link)

    # 목표 수량 도달 시 루프 종료
    if len(product_links) >= target_count:
        break

    # 스크롤 다운
    body = driver.find_element(By.TAG_NAME, 'body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(scroll_pause_time)

    # 상품 수 출력 (디버깅용)
    print(f"중복 없는 상품 수: {len(product_links)}")

# 결과 출력
print("수집된 상품 링크:")
for link in product_links:
    print(link)
print(f"총 {len(product_links)}개의 상품 링크 수집 완료")

# 저장할 폴더
os.makedirs(f"./연인/기념일", exist_ok=True)

info_dic = {}
i = 0

for link in tqdm(product_links):
    time.sleep(2)
        # info 
    try:
        info = crawl_info(link) 
    except Exception as e:
        print(e)
        # i += 1
        continue
    time.sleep(1)
    # review
    satisfaction, review = crawl_review(link)

    # 데이터 저장
    info['satisfaction'] = satisfaction
    info['link'] = link


    # 리뷰가 없는 상품은 건너뜁니다.
    if info['num_review'] == '0' or info['num_review'] == 0:
        # i += 1
        continue


    # info_dic에 상품 정보 저장
    info_dic[info['name']] = info

    # 각 상품별로 리뷰 모음 딕셔너리가 생성됩니다.
    # 저장 -> csv
    review_df = pd.DataFrame(review)
    review_df['product'] = info['name']
    review_df['link'] = info['link']

    review_df.to_csv(f"연인/기념일/{i}.csv", index=False)
    i += 1

    # 100개 크롤링 후 종료
    if len(info_dic) >= 40:
        break

info_df = pd.DataFrame(info_dic).T
info_df.to_csv(f"연인/기념일/metadata.csv", index=False)

중복 없는 상품 수: 20
중복 없는 상품 수: 20
중복 없는 상품 수: 20
중복 없는 상품 수: 20
중복 없는 상품 수: 20
수집된 상품 링크:
https://gift.kakao.com/product/749180?banner_id=747&campaign_code=null
https://gift.kakao.com/product/6376669?banner_id=747&campaign_code=null
https://gift.kakao.com/product/7462331?banner_id=747&campaign_code=null
https://gift.kakao.com/product/37794?banner_id=747&campaign_code=null
https://gift.kakao.com/product/5827254?banner_id=747&campaign_code=null
https://gift.kakao.com/product/10483683?banner_id=747&campaign_code=null
https://gift.kakao.com/product/2324542?banner_id=747&campaign_code=null
https://gift.kakao.com/product/9457951?banner_id=747&campaign_code=null
https://gift.kakao.com/product/10777533?banner_id=747&campaign_code=null
https://gift.kakao.com/product/10669211?banner_id=747&campaign_code=null
https://gift.kakao.com/product/10969889?banner_id=747&campaign_code=null
https://gift.kakao.com/product/10674544?banner_id=747&campaign_code=null
https://gift.kakao.com/product/9664955?banner_id

  2%|▎         | 1/40 [00:09<06:26,  9.92s/it]

리뷰 수: 5


  5%|▌         | 2/40 [00:19<06:14,  9.86s/it]

리뷰 수: 5


...

리뷰 수: 22


 98%|█████████▊| 39/40 [07:56<00:11, 11.32s/it]

리뷰 수: 1
리뷰 수: 20
2024-10-18 00:00:00 2024-09-30 00:00:00
리뷰 수: 20
2024-10-18 00:00:00 2024-09-30 00:00:00


 98%|█████████▊| 39/40 [08:08<00:12, 12.54s/it]

리뷰 수: 21
